In [7]:
# OPTIMIZACION DEL BUTTERWORTH

import yfinance as yf
import pandas as pd 
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from math import cos,pi
from scipy.optimize import newton

# ALPHA PARA LA EMA (con newton):
def alpha_ema(wc):
    wc = wc*pi
    B = 2*(1-cos(wc)); C = 2*(cos(wc)-1)
    func = lambda alpha: alpha**2 + B*alpha + C 
    deriv = lambda alpha: 2*alpha + B
    alpha_0 = 0.5
    return round(newton(func, alpha_0, deriv),3)

# FILTRO EMA Y BANDAS DE BOLLINGER

def ema(close,wc):
    alpha = alpha_ema(wc)
    y = []

    for n in range(len(close)):
        if n == 0:
            y.append(close[0])
        else:
            y_n = alpha*close[n] + (1-alpha)*y[n-1]
            y.append(y_n)
    return y

def T_sma(wc):

    # Metodo de Newton para obtener el perido de la SMA a partir de la frecuencia de corte (para las bollinger)
    # Cuando usamos una SMA es facil obtener las BB porque para la media de la std se usa el mismo periodo que la sma
    # Pero cuando estamos trabajando con la wc no sabemos el periodo. Una alternativa es programar las BB a partir de la wc
    wc = wc*pi
    func = lambda N: np.sin(wc*N/2) - (N/np.sqrt(2))*(np.sin(wc/2))
    deriv = lambda N: (wc/2)*np.cos(wc*N/2) - (1/np.sqrt(2))*np.sin(wc/2)
    N_0 = pi/wc  
    return int(np.round(newton(func, N_0, deriv)))

def Vol_L(close,wc,smooth,num_std = 2):
    return smooth - num_std*pd.Series(close).rolling(T_sma(wc)).std()

def Vol_U(close,wc,smooth,num_std = 2):
    return smooth + num_std*pd.Series(close).rolling(T_sma(wc)).std()

# La desviacion estandar se calcula usando una media movil pero es el periodo el que se define con el metodo de newton al no saber T

In [8]:
# SISTEMA 
class strat_BB(Strategy):

    wc = 0.05 # inicializamos a 0.1 
    N = 2 # std dev from mean 
    sl_pct = 0.015

    def init(self):

        # Parameters 
        close = self.data.Close
        self.smooth = ema(close,self.wc)

        # Indicators 
        self.upper_band = self.I(Vol_U, close, self.wc, self.smooth, self.N)
        self.lower_band = self.I(Vol_L, close, self.wc, self.smooth, self.N)

    def next(self):

        price = self.data.Close[-1]

        # Buy   
        if crossover(self.lower_band,self.data.Close):
            self.position.close()
            self.buy(sl = price*(1-self.sl_pct))
            
        # Sell
        elif crossover(self.data.Close,self.upper_band):
            self.position.close()
            self.sell(sl = price*(1+self.sl_pct))

In [11]:
# DATA IMPORT AND BACKTESTING 

xls = r'C:\Users\Usuario\Desktop\TFG\S&P 500 Companies (Standard and Poor 500).xlsx'
file_df = pd.read_excel(xls)
all_tickers = file_df['Symbol']
tickers = all_tickers

mu = []; sigma = []; rets = []; ntrades = []

for i in range(len(tickers)):
    try:
        df = yf.download(tickers[i],  start = '2000-01-01', end = '2023-01-01', progress=False)
        iCash = 10000
        com = 0.00

        #BACKTESTING 
        bt = Backtest(df, strat_BB, cash = iCash, commission = com, exclusive_orders = True)

        # OUTPUTS
        output = bt.run()
        rets.append(output['# Trades'])
        mu.append(output['Return (Ann.) [%]'])
        sigma.append(output['Volatility (Ann.) [%]'])
        ntrades.append(output['# Trades'])

    except: 
        rets.append(None); mu.append(None); sigma.append(None); ntrades.append(None)

# RESULTS
res = pd.DataFrame({'Ticker':tickers, 'Annualized Returns (%)':mu, 'Annualized Volatility (%)':sigma, 'Trades':ntrades}).dropna()
res = res[res['Trades'] != 0]
res = res[res['Annualized Volatility (%)'] < 200]
mean_values = res.mean()
mean_row = pd.DataFrame(mean_values).T
res = pd.concat([res, mean_row]).reset_index(drop=True)
res.loc[res.index[-1], 'Ticker'] = 'MEAN'

#bt.plot(plot_volume = False)
#res.to_excel(r'C:\Users\Usuario\Desktop\TFG\Resultados Excel\Segunda Simulacion\Comparativa SMA vs BUTT.xlsx', index = False)
res

C:\Users\Usuario\AppData\Local\Temp\ipykernel_14832\1531149733.py:17: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, strat_BB, cash = iCash, commission = com, exclusive_orders = True)



1 Failed download:
- ARG: No data found for this date range, symbol may be delisted

1 Failed download:
- ALXN: No data found, symbol may be delisted

1 Failed download:
- AGN: No data found, symbol may be delisted

1 Failed download:
- APC: No data found, symbol may be delisted

1 Failed download:
- APOL: No data found for this date range, symbol may be delisted

1 Failed download:
- AVP: No data found, symbol may be delisted

1 Failed download:
- BLL: No data found, symbol may be delisted

1 Failed download:
- BCR: No data found for this date range, symbol may be delisted

1 Failed download:
- BBT: No data found, symbol may be delisted

1 Failed download:
- BBBY: No data found, symbol may be delisted

1 Failed download:
- BRK.B: No data found, symbol may be delisted


C:\Users\Usuario\AppData\Local\Temp\ipykernel_14832\1531149733.py:17: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, strat_BB, cash = iCash, commission = com, exclusive_orders = True)



1 Failed download:
- BRCM: No data found for this date range, symbol may be delisted

1 Failed download:
- BF.B: No data found for this date range, symbol may be delisted

1 Failed download:
- CVC: No data found for this date range, symbol may be delisted

1 Failed download:
- COG: No data found, symbol may be delisted

1 Failed download:
- CAM: No data found for this date range, symbol may be delisted

1 Failed download:
- CFN: No data found for this date range, symbol may be delisted

1 Failed download:
- CBG: No data found for this date range, symbol may be delisted

1 Failed download:
- CBS: No data found, symbol may be delisted

1 Failed download:
- CELG: No data found, symbol may be delisted

1 Failed download:
- CTL: No data found, symbol may be delisted

1 Failed download:
- CERN: No data found, symbol may be delisted

1 Failed download:
- CTXS: No data found, symbol may be delisted

1 Failed download:
- COH: No data found for this date range, symbol may be delisted

1 Failed 

C:\Users\Usuario\AppData\Local\Temp\ipykernel_14832\1531149733.py:17: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, strat_BB, cash = iCash, commission = com, exclusive_orders = True)


...


1 Failed download:
- PCLN: No data found for this date range, symbol may be delisted

1 Failed download:
- QEP: No data found, symbol may be delisted

1 Failed download:
- RTN: No data found, symbol may be delisted

1 Failed download:
- RHT: No data found, symbol may be delisted

1 Failed download:
- RAI: No data found for this date range, symbol may be delisted

1 Failed download:
- RDC: No data found, symbol may be delisted

1 Failed download:
- SWY: No data found for this date range, symbol may be delisted

1 Failed download:
- SNDK: No data found for this date range, symbol may be delisted

1 Failed download:
- SHLD: Data doesn't exist for startDate = 946681200, endDate = 1672527600

1 Failed download:
- SIAL: No data found for this date range, symbol may be delisted

1 Failed download:
- STJ: No data found for this date range, symbol may be delisted

1 Failed download:
- SPLS: No data found for this date range, symbol may be delisted

1 Failed download:
- STI: No data found, symb

C:\Users\Usuario\AppData\Local\Temp\ipykernel_14832\1531149733.py:17: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, strat_BB, cash = iCash, commission = com, exclusive_orders = True)
c:\Users\Usuario\anaconda3\envs\pyenv\lib\site-packages\backtesting\_stats.py:114: RuntimeWarning: overflow encountered in double_scalars
  s.loc['Volatility (Ann.) [%]'] = np.sqrt((day_returns.var(ddof=int(bool(day_returns.shape))) + (1 + gmean_day_return)**2)**annual_trading_days - (1 + gmean_day_return)**(2*annual_trading_days)) * 100  # noqa: E501



1 Failed download:
- TMK: No data found, symbol may be delisted

1 Failed download:
- TSS: No data found, symbol may be delisted

1 Failed download:
- TYC: No data found for this date range, symbol may be delisted

1 Failed download:
- UTX: No data found, symbol may be delisted

1 Failed download:
- VAR: No data found, symbol may be delisted

1 Failed download:
- VIAB: No data found, symbol may be delisted

1 Failed download:
- WAG: No data found for this date range, symbol may be delisted

1 Failed download:
- WPO: No data found for this date range, symbol may be delisted

1 Failed download:
- WLP: No data found for this date range, symbol may be delisted

1 Failed download:
- WFM: No data found for this date range, symbol may be delisted

1 Failed download:
- WIN: No data found, symbol may be delisted

1 Failed download:
- WPX: No data found, symbol may be delisted

1 Failed download:
- WYN: No data found for this date range, symbol may be delisted

1 Failed download:
- XLNX: No dat

C:\Users\Usuario\AppData\Local\Temp\ipykernel_14832\1531149733.py:33: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_values = res.mean()


,Ticker,Annualized Returns (%),Annualized Volatility (%),Trades
0,MMM,-2.421363,9.205626,132.000000
1,ACE,-9.187154,19.368425,6.000000
2,ABT,-1.358335,9.676964,133.000000
3,ANF,1.832784,14.060595,152.000000
4,ACN,-0.663602,12.098296,142.000000
...,...,...,...,...
388,XRX,-1.613641,13.539614,145.000000
389,XYL,3.880492,13.098863,54.000000
390,YUM,-2.814212,9.862463,146.000000
391,ZION,0.383862,13.928950,136.000000
